# **non overlapping simulations**

In [2]:
#bash statemnt format 
# Description_of_sim,  Number_gens, where to i=find initial dna,   where to find initial dnamap 

# IMPORT STATEMENTS~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import random
from random import randint
from typing import List, Dict
from numpy.random import choice
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy
from tqdm import tqdm
import copy 
import json

In [3]:

#MAKING STATIC OBJECTS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#mutability model 
model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))

In [4]:
#FUNCTIONS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def base_to_mutate(DNA, current_weights): 
    """
    (DNA: str, weights: Dict) -> int
    
    Will return the index of a random choice of a base that will be mutated based on probabilities given by the
    weights dictionary (i.e. the model)
    ex. "ATCGTA" --> index 3 ("G") will mutate
    """
    #DNA STR --> WEIGHTS LIST INDEXING DOUBLE CHECK 
    indices = []
    indices = list(range(len(DNA)))                        
    assert (len(indices)) == (len(current_weights)) 

    # NORMALIZE THE POPULATION OF WEIGHTS    
    total_freq = sum(current_weights) 
    normalized_weights = copy.copy(current_weights)
    for index, value in enumerate(current_weights):
        normalized_weights[index] = value/total_freq 

    # DRAW THE INDEX OF THE BASE THAT WILL BE MUTATED
    base_index = choice(indices, p=normalized_weights)  
    
    # RETURN THE INDEX
    return base_index

def triplet_to_turn_into(i, model, DNA):
    """
    (i: int, weights: Dict, DNA: str) -> str (the base A/T/C/G)
    """
    # GRAB THE TRIPLET AND COMPLEMENT IT IF NEED BE.
    triplet = DNA[i] 
            
    # GENERATE A LIST OF THE TRIPLETS AND WEIGHTS OF THE THREE TRIPLETS THE CHOSEN TRIPLET CAN TURN INTO 
    triplets = list(key for key in model[triplet][1].keys())# ex. "ATC" --> ["AAC", "AGC", "ACC"] (triplets)
    current_weights_to_turn_into = list(value for value in model[triplet][1].values()) # ex. "ATCGTA" --> [0.3, 0.5, 0.1, 0.1] (weights of triplet base changes) (these are already normalized!)
    
    # MAKE AND RETURN THE CHOICE OF BASE TO TURN INTO.
    return choice(triplets, p = current_weights_to_turn_into) # <-- the [1] will select the central base (middle N base out of strign 3)

def mutate_chromosome(DNA, ab_index, mtrip):
    """
    (DNA:list, ab_index:number, site chosen, mb:str) -> list (mutated DNA)
    Will return a new DNA list mutated to a new base as given by the previous functions.
    """
    mutated_DNA = copy.copy(DNA)
    mutated_DNA[ab_index] = mtrip
    
    return mutated_DNA

def current_frequencies(DNA, model, index, current_weights,mtrip): #and by weights, she means the "model" dictionary 
    """
    (DNA: str, weights list, index = chosen site) -> updated weights List 
    only chnages the 3 bases around the index chosen 
    """    
    #GET THE WEIGHTS FOR EACH TRIPLET AND UPDATE WEIGHTS (LIST) OBJECT 
    new_weights = copy.copy(current_weights)
    new_weights[index]= model[mtrip][0]
    return new_weights



In [12]:
ntrial = 10 
ntrips = int(1000) 
prop_to_print = [0,0.5,1,1.5,2]   #outputing a file at different mutation coverages to compare 
gens_to_print = [int(i*ntrips) for i in prop_to_print]
gens_to_print[-1] = gens_to_print[-1]-1 #the number of genns last to ngen -1, so need to sub one here to make sure it prints at the end 

for k in tqdm(range(1,ntrial+1)): 
    
    #set up for simulation 
    DNA = list(choice(list(model.keys()),size =ntrips))
    weights = []
    for trip in DNA: 
        weights.append(model[trip][0])
    for j,triplet in enumerate(DNA):                                   #makign sure i got the weights right 
            if model[triplet][0] != weights[j]: 
                print("fuck")

    # running the simulation 
    for i in range(len(DNA)*2):
        ab_index = base_to_mutate(DNA,weights)                          # picking the triple to mutate 
#         mtrip = triplet_to_turn_into(ab_index, model, DNA)              #selecting triplet to turn into 
        mtrip = choice(list(model.keys()))                              #random into triplet 
        for j,triplet in enumerate(DNA):                                #double check the weights match 
            if model[triplet][0] != weights[j]: 
                print("ab_index, mtrip functions.",triplet, j, weights[j])
        DNA = mutate_chromosome(DNA, ab_index,mtrip)                    #mutate dna 
        weights[ab_index]= model[mtrip][0]                              #updatign weights
        for j,triplet in enumerate(DNA):                                #double check the weights match 
            if model[triplet][0] != weights[j]: 
                print("MC & UW",j,triplet,"weights", weights[j],"model val",model[triplet][0]) 
    
        if i in gens_to_print: 
            with open("data/nonOverlap_randInto/nonOverlap_randInto_trial{k}_DNAfinal_{n}trips_gen{i}.txt".format(k=k,n=ntrips,i=i), "w") as outfile:  #write conditionsa to file 
                    json.dump(DNA, outfile)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.66s/it]


In [11]:
choice(list(model.keys()))

'TAA'

# **preping the non overlapping results for plotting**

In [21]:
trips_32 = []
for base1 in ["A","T","C","G"]: 
    for base2 in ["C","T"]: 
        for base3 in  ["A","T","C","G"]: 
            trips_32.append(base1+base2+base3)

In [22]:
def revComp(triplet): 
    conversion_dict = {"A":"T","T":"A","C":"G","G":"C"} #dictionary that contaisn the reverse complement of each base 
    if triplet[1] in ["A","G"]: # i personally like to remove A&Gs 
        rc_triplet = str()      #set up the new triplet ot be created 
        for base in triplet: 
            rc_triplet = conversion_dict[base]+rc_triplet #adds the reverse complemented bases to the rc_triplet in reverse order 
        return rc_triplet
    else: 
        return triplet    #if the middle base is C or T, keep the triplet as is 

In [33]:
trip_sum_dict = {"triplet":[],"mut":[],"log10mut":[]}
for gen in gens_to_print: 
    trip_sum_dict["avCounts_gen{g}".format(g=int(gen))] = []
    trip_sum_dict["stderrCounts_gen{g}".format(g=int(gen))]=[]

for triplet in trips_32: 
    if triplet[1] in ["C","T"]:
        trip_sum_dict["triplet"].append(triplet)
        trip_sum_dict["mut"].append(model[triplet][0])
        trip_sum_dict["log10mut"].append(np.log10(model[triplet][0]))
        for gen in gens_to_print: 
            triplet_list = []
            for k in range(2,ntrial+1): 
        #         dna_init = json.load(open("data/nonOverlap_trial{k}_DNA_333trips_gen0.txt"))
                filename = "data/nonOverlap_randInto/nonOverlap_randInto_trial{k}_DNAfinal_{n}trips_gen{i}.txt".format(k=k,n=ntrips,i=int(gen))
                dna_final= json.load(open(filename))
                triplet_list.append(dna_final.count(triplet))
                triplet_list.append(dna_final.count(revComp(triplet)))
            trip_sum_dict["avCounts_gen{g}".format(g=gen)].append(np.mean(triplet_list))
            trip_sum_dict["stderrCounts_gen{g}".format(g=gen)].append(np.std(triplet_list)/np.sqrt(ntrial))


In [34]:
trip_sum_df = pd.DataFrame.from_dict(trip_sum_dict)

In [35]:
trip_sum_df.to_csv("data/nonOverlap_randInto/nonoverlapData_randInto_2021_12_11.csv")

In [1]:
!pwd

/research/projects/hsapiens/mutation_equilibrium/nonoverlapSimulation
